# ネガ・ポジ分析
ネガ・ポジ分析では主に人の発言や発想などが、前向き（ポジティブ）か後ろ向き（ネガティブ）かを分析可能</br>
ネガ・ポジ分析は「感情分析」（Sentiment Analysis）と呼ばれる技術の一種</br>
これは、文章などに含まれる評価・感情に関する表現を抽出して、文章中の感情を解析する技術などを指す


## 極性辞書
ネガ/ポジのことを「極性」と呼び、極性辞書とは単語ごとに極性を付けてまとめたものを指す

- PN Table
    - 極性辞書は人力で大量の単語に極性を当てたわけではなく、
      少量の極性情報をもつ単語をベースに関連度の高い単語に-1~+1までの点数を割り振ることで作られている
- 日本語評価極性辞書
    - 東北大の乾・岡崎研究室のページで公開されている
    - ネガ・ポジ以外にニュートラルというタグをつけることで、辞書に含まれている単語の極性バランスを取っている
- Polar Phrase Dictionary
    - Yahoo!JAPAN研究所

In [2]:
import pandas as pd
# pn_df = pd.read_csv('./6020_negative_positive_data/data/pn_ja.dic',\
#                     sep=':',
#                     encoding='utf-8',
#                     names=('Word','Reading','POS', 'PN')
#                    )

#  PNTableを出力
# print(pn_df)


# 極性辞書を用いたネガ・ポジ分析
## 形態素解析
形態素解析とは文章の中から最小の単位となる単語に分割する作業</br>
形態素解析を行うことで、極性辞書に対応する単語を見つけることができる</br>
今回はMeCabを使った形態素解析を行い、文章を読み込みやすい形に変える

In [10]:
import MeCab
mecab = MeCab.Tagger('')
title = open('./6020_negative_positive_data/data/aidokushono_insho.txt')
file = title.read()
title.close()

print(mecab.parse(file).split("\n")[0])

愛読	名詞,サ変接続,*,*,*,*,愛読,アイドク,アイドク


## 形態素解析のリスト化
解析結果をリスト化することで他の処理をしやすい形にする</br>
MeCabで形態素解析を行うと、最後の行が"空白"、最後から2番目の行が"EOS"となる</br>
その2行は使わないので削除する処理を行う</br>
解析結果のそれぞれの行では単語の後にタブ、他の情報はカンマで区切っている</br>


In [7]:
import MeCab
import pandas as pd
import re

mecab = MeCab.Tagger('')
title = open('./6020_negative_positive_data/data/aidokushono_insho.txt')
file = title.read()
title.close()
def get_diclist(file):
    parsed = mecab.parse(file)
    # 解析結果を改行ごとに区切る
    lines = parsed.split('\n')
    # 後ろ2行を削除した新しいリストを作る
    lines = lines[0:-2]
    
    #解析結果のリストを作成
    diclist = []
    for word in lines:
        # タブとカンマで区切ったデータを作成
        data = re.split('\t|,',word)  
        datalist = {'BaseForm':data[0]}
        diclist.append(datalist)
    return(diclist)

wordlist = get_diclist(file)
print(wordlist[0])

{'BaseForm': '愛読'}


## 解析結果にネガ・ポジ判定を行う
極性辞書(PNTable)を読み込み、解析結果のリストと照らし合わせることで出現した単語に極性を与えることができる</br>
PNTableから単語と極性値のみの辞書を作成</br>
新しくなったPNTableに存在している単語、極性値の新しいリストを作成</br>

In [11]:
import pandas as pd
import MeCab
import pandas as pd
import re

mecab = MeCab.Tagger('')
title = open('./6020_negative_positive_data/data/aidokushono_insho.txt')
file = title.read()
title.close()

# 辞書を読み込む
pn_df = pd.read_csv('./6020_negative_positive_data/data/pn_ja.dic',\
                    sep=':',
                    encoding='utf-8',
                    names=('Word','Reading','POS', 'PN')
                   )

# PNTableを単語と極性値のみのdict型に変更
word_list = list(pn_df['Word'])
pn_list = list(pn_df['PN'])
pn_dict = dict(zip(word_list, pn_list))

# 解析結果のリストからPNTableに存在する単語を抽出
def add_pnvalue(diclist_old):
    diclist_new = []
    for word in diclist_old:
        baseword = word['BaseForm']        
        if baseword in pn_dict:
            # PNTableに存在していればその単語の極性値を追加
            # print(baseword)
            pn = pn_dict[baseword]
            
        else:
            # 存在していなければnotfoundを明記
            pn = 'notfound'
        # 極性値を持つ単語に極性値を追加
        word['PN'] = pn
        diclist_new.append(word)
    return(diclist_new)

def get_diclist(file):
    parsed = mecab.parse(file)
    # 解析結果を改行ごとに区切り
    lines = parsed.split('\n')
    # 後ろ2行を削除した新しいリストを作成
    lines = lines[0:-2]
    
    #解析結果のリストを作成
    diclist = []
    for word in lines:
        # タブとカンマで区切ったデータを作成
        data = re.split('\t|,',word)  
        datalist = {'BaseForm':data[0]}
        diclist.append(datalist)
    return(diclist)

wordlist = get_diclist(file)
pn_list = add_pnvalue(wordlist)

print(pn_list[0:3])

[{'BaseForm': '愛読', 'PN': -0.207109}, {'BaseForm': '書', 'PN': -0.688965}, {'BaseForm': 'の', 'PN': 'notfound'}]


# RNNを使ったTwitterのネガ・ポジ分析 前半
## RNNとは
極性辞書では文章の文脈を読んでネガ・ポジを判断することが難しく、適切な分析ができないことがある</br>
そこで、再帰型ニューラルネットワーク(RNN)を使って文章の流れからネガ・ポジ分析を行う手法について学ぶ</br>
RNNは以前に計算された情報を記憶して学習することができるので、文章の次に来る単語を予測したり、単語の出現確率などから機械翻訳に使われている</br>


## Twitterのネガ・ポジ分析
Twitterでは140文字という文字制限があるので、短い文章でユーザーが発信している</br>
大量のデータを手に入れることができるので、ネガ・ポジ分析を始め様々な自然言語処理の分析に使われている</br>
データは米Figure Eightが配布しているAirline Twitter sentimentを使う

https://appen.com/open-source-datasets/


In [15]:
import pandas as pd

Tweet = pd.read_csv('./6020_negative_positive_data/data/Airline-Sentiment-2-w-AA.csv', encoding='cp932')
# tweetData = Tweet.loc[:, ["text", "airline_sentiment"]]
tweetData = Tweet[["text", "airline_sentiment"]]

print(tweetData)

                                                    text airline_sentiment
0                    @VirginAmerica What @dhepburn said.           neutral
1      @VirginAmerica plus you've added commercials t...          positive
2      @VirginAmerica I didn't today... Must mean I n...           neutral
3      @VirginAmerica it's really aggressive to blast...          negative
4      @VirginAmerica and it's a really big bad thing...          negative
...                                                  ...               ...
14590  @AmericanAir thank you we got on a different f...          positive
14591  @AmericanAir leaving over 20 minutes Late Flig...          negative
14592  @AmericanAir Please bring American Airlines to...           neutral
14593  @AmericanAir you have my money, you change my ...          negative
14594  @AmericanAir we have 8 ppl so we need 2 know h...           neutral

[14595 rows x 2 columns]


## データベースの作成
### 頻出単語の削除
RNNでは単語の関連性を分析するので、頻出する単語を削除する必要はない</br>
Iやwhatなどの頻出する単語をストップワードと呼ぶ</br>
Googleの検索エンジンではストップワードを検索対象外にすることで他の単語同士の関連度を高めている</br>
Twitterで返信を意味する@やflightという単語が頻出しているので、それらの単語を削除したデータを作成


In [20]:
import nltk
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords

# エラーが出たら↓をコメントアウト
# nltk.download('stopwords')

# Tweetデータの読み込みます
Tweet = pd.read_csv('./6020_negative_positive_data/data/Airline-Sentiment-2-w-AA.csv', encoding='cp932')
tweetData = Tweet.loc[: 100,['text','airline_sentiment']] # メモリの都合上、読み込むデータを制限しています。

# 英語Tweetの形態素解析を行う
def tweet_to_words(raw_tweet):
    # a~zまで始まる単語を空白ごとに区切ったリスト作成
    letters_only = re.sub("[^a-zA-Z@]", " ", raw_tweet) 
    words = letters_only.lower().split()
    # '@'と'flight'が含まれる文字とストップワードを削除
    stops = set(stopwords.words("english"))  
    meaningful_words =  [w for w in words if not w in stops and not re.match("^[@]", w) and not re.match("flight", w)] #条件1 and not #条件2 ] 
    return(" ".join(meaningful_words)) 

cleanTweet = tweetData['text'].apply(lambda x: tweet_to_words(x))
print(cleanTweet)

0                                                   said
1                plus added commercials experience tacky
2                 today must mean need take another trip
3      really aggressive blast obnoxious entertainmen...
4                                   really big bad thing
                             ...                        
96     check add bag website working tried desktop mo...
97     let scanned passengers leave plane told someon...
98                    phone number find call reservation
99     anyone anything today website useless one answ...
100    trying add boy prince ressie sf thursday lax h...
Name: text, Length: 101, dtype: object


### 単語のデータベースを作成
どの単語がネガ・ポジに影響を与えるか調べるために、一度単語のデータベースを作成</br>
データベースを使って、単語の出現頻度やネガ・ポジのタグづけを行う


In [23]:
import nltk
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords

# エラーが出たら↓をコメントアウト
# nltk.download('stopwords')

Tweet = pd.read_csv('./6020_negative_positive_data/data/Airline-Sentiment-2-w-AA.csv', encoding='cp932')
tweetData = Tweet.loc[: 100,['text','airline_sentiment']] # メモリの都合上、読み込むデータを制限している

# 英語Tweetの形態素解析を行う
def tweet_to_words(raw_tweet):
    # a~zまで始まる単語を空白ごとに区切ったリストを作成
    letters_only = re.sub("[^a-zA-Z@]", " ",raw_tweet) 
    words = letters_only.lower().split()
    # '@'と'flight'が含まれる文字とストップワードを削除
    stops = set(stopwords.words("english"))  
    meaningful_words = [w for w in words if not w in stops and not re.match("^[@]", w) and not re.match("flight",w)] 
    return(" ".join( meaningful_words )) 

cleanTweet = tweetData['text'].apply(lambda x: tweet_to_words(x)) 
#データベースを作成
all_text = ' '.join(cleanTweet)
words = all_text.split()
print(words[0])

said


### 単語の数値化
単語の出現回数をベースにそれぞれの単語に数値のタグづけを行う</br>
今回学習に使うcleanTweetの文字列を数値化して新しいリストを作成

In [25]:
import nltk
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from collections import Counter

# エラーが出たら↓をコメントアウトしてください
# nltk.download('stopwords')

Tweet = pd.read_csv('./6020_negative_positive_data/data/Airline-Sentiment-2-w-AA.csv', encoding='cp932')
tweetData = Tweet.loc[: 100,['text','airline_sentiment']] # メモリの都合上、読み込むデータを制限しています。

def tweet_to_words(raw_tweet):
    
    # a~zまで始まる単語を空白ごとに区切ったリストをつくります
    letters_only = re.sub("[^a-zA-Z@]", " ",raw_tweet) 
    words = letters_only.lower().split()
    
    # '@'と'flight'が含まれる文字とストップワードを削除します
    stops = set(stopwords.words("english"))  
    meaningful_words = [w for w in words if not w in stops and not re.match("^[@]", w) and not re.match("flight",w)] 
    return( " ".join( meaningful_words )) 

cleanTweet = tweetData['text'].apply(lambda x: tweet_to_words(x))

#データベースを作成します
all_text = ' '.join(cleanTweet)
words = all_text.split()

# 単語の出現回数をカウントします
counts = Counter(words)
# 降順に並べ替えます
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}

# 新しいリストに文字列を数値化したものを格納します
tweet_ints = []
for each in cleanTweet:
    tweet_ints.append([vocab_to_int[word] for word in each.split()])
    
print(tweet_ints)

[[143], [144, 66, 145, 19, 146], [34, 147, 148, 8, 35, 67, 12], [13, 149, 150, 151, 152, 153, 154, 20, 155, 156], [13, 157, 68, 69], [158, 36, 159, 70, 160, 13, 68, 69, 9, 37], [161, 162, 71, 4, 14, 72, 163, 164, 73, 38], [13, 165, 166, 167, 168, 169, 170, 171, 74, 1, 172, 173], [174], [39, 175, 40, 176, 41], [5, 177, 178, 179, 75, 180, 181, 182], [76, 183, 184, 185, 186, 187, 188], [6, 77, 189, 190, 191, 12, 20, 192, 193, 78, 12, 194, 79], [9, 195, 196, 197, 198, 35, 199, 38, 80, 2, 1, 200], [42], [43, 201, 202, 44, 203], [45, 10, 204, 81, 3, 205, 206, 207, 6, 208, 46, 82, 209], [47, 83, 43, 21, 48, 210, 211, 22, 212, 213, 214, 215, 216, 217, 7], [9], [5, 36, 218, 49, 84, 219, 50, 85, 14], [10, 220, 221, 51, 222, 223, 70, 224, 225], [52, 226, 2, 1, 227, 228], [52, 229, 230, 231, 41, 232], [233, 84, 53, 234, 235, 236, 237, 238, 54], [86, 239, 87, 240, 87, 241, 86, 242, 22, 38, 85, 243, 244], [88, 245, 246, 89, 51, 90, 91, 92, 247], [93, 94, 248, 249, 250, 251, 252, 94, 55, 5, 253, 254,

### ネガ・ポジの数値化
文章ごとに与えられたネガ・ポジの評価を数値化</br>
今回はネガティブ=0, ポジティブ=1, ニュートラル=2に変換</br>
ネガ・ポジ数値は単語から生成された文章を学習する際に使用

In [26]:
import numpy as np
import pandas as pd

# Tweetデータの読み込みます
Tweet = pd.read_csv('./6020_negative_positive_data/data/Airline-Sentiment-2-w-AA.csv', encoding='cp932')
tweetData = Tweet.loc[:,['text','airline_sentiment']]

# tweetDataのnegativeを0, positiveを1,それ以外の文字列(neutral)を2に変換します
labels = np.array([0 if each == 'negative' else 1 if each == 'positive' else 2 for each in tweetData['airline_sentiment'][:]]) 
print(labels)

[2 1 2 ... 2 0 2]


### 列の数を揃える
作成したtweet_intsではweetごとにバラバラな単語数が格納されている</br>
学習する際にリストの列を揃える必要がある</br>
cleanTweetの処理により単語数が0になってしまった行をそれぞれのリストから削除


In [27]:
from collections import Counter
import nltk
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from collections import Counter

# エラーが出たら↓をコメントアウトしてください
nltk.download('stopwords')

def tweet_to_words(raw_tweet):
    
    # a~zまで始まる単語を空白ごとに区切ったリストをつくります
    letters_only = re.sub("[^a-zA-Z@]", " ",raw_tweet) 
    words = letters_only.lower().split()
    
    # '@'と'flight'が含まれる文字とストップワードを削除します
    stops = set(stopwords.words("english"))  
    meaningful_words = [w for w in words if not w in stops and not re.match("^[@]", w) and not re.match("flight",w)] 
    return( " ".join( meaningful_words )) 

# Tweetデータの読み込みます
Tweet = pd.read_csv('./6020_negative_positive_data/data/Airline-Sentiment-2-w-AA.csv', encoding='cp932')
tweetData = Tweet.loc[:,['text','airline_sentiment']]

cleanTweet = tweetData['text'].apply(lambda x: tweet_to_words(x))

# Tweetのnegative/positiveの文字列を数字に変換します
labels = np.array([0 if each == 'negative' else 1 if each == 'positive' else 2 for each in tweetData['airline_sentiment'][:]]) 
#データベースを作成します
all_text = ' '.join(cleanTweet)
words = all_text.split()

# 単語の出現回数をカウントします
counts = Counter(words)
# 降順に並べ替えます
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}

# 新しいリストに文字列を数値化したものを格納します
tweet_ints = []
for each in cleanTweet:
    tweet_ints.append([vocab_to_int[word] for word in each.split()])

# Tweetの単語数を調べます
tweet_len = Counter([len(x) for x in tweet_ints])
seq_len = max(tweet_len)
print("Zero-length reviews: {}".format(tweet_len[0]))
print("Maximum review length: {}".format(max(tweet_len)))

# cleanTweetで単語数が0になってしまった行をそれぞれのリストから削除します
tweet_idx  = [idx for idx,tweet in enumerate(tweet_ints) if len(tweet) > 0]
labels = labels[tweet_idx]
tweetData = tweetData.iloc[tweet_idx]
tweet_ints = [tweet for tweet in tweet_ints if len(tweet) > 0]

# 列数を揃えるためにi行ごとの数値化した単語を右から書いたフレームワークを作ります
features = np.zeros((len(tweet_ints), seq_len), dtype=int)
for i, row in enumerate(tweet_ints):
    features[i, -len(row):] = np.array(row)[:seq_len]

print(features)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/k-kakimoto/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Zero-length reviews: 34
Maximum review length: 107
[[    0     0     0 ...     0     0   121]
 [    0     0     0 ...  2226   106  5734]
 [    0     0     0 ...    73    69   100]
 ...
 [    0     0     0 ...   326   146 12814]
 [    0     0     0 ...  1288    48  2373]
 [    0     0     0 ...   472    62    92]]


# RNNを使ったTwitterのネガ・ポジ分析（後半）
## LSTMとは
LSTM(ロングショートタームメモリー)はRNNにおける記憶、つまり中間層を担っているシステム</br>
従来RNNの構想は系列データを取り扱うために研究されていましたが実現可能なアルゴリズムではなかった</br>
LSTMはRNNを実際に利用するために開発された手法であり、長期間の時間依存を保持することができる


## 学習データセットの作成
前半で作成したデータを元に、学習で使うデータセットを作成</br>
train_xでは単語の数値化をしたもの、train_yではネガ・ポジを数値化したものをそれぞれ作成</br>
test_x, test_yは共に学習結果の精度を確認するために使用</br>

LSTMで学習させるデータは(サンプル数, シーケンス長, 次元数) の形に変形</br>
。シーケンス長とは、モデルがステップ毎に学習するデータの数を指す

In [28]:
#chapter2のおさらい
import nltk
import numpy as np
import pandas as pd
import re
from collections import Counter
from nltk.corpus import stopwords
from tensorflow.keras.utils import to_categorical

# エラーが出たら↓をコメントアウトしてください
nltk.download('stopwords')

# Tweetデータを読み込みます
Tweet = pd.read_csv('./6020_negative_positive_data/data/Airline-Sentiment-2-w-AA.csv', encoding='cp932')
tweetData = Tweet.loc[:,['text','airline_sentiment']]

# 英語Tweetの形態素解析を行います
def tweet_to_words(raw_tweet):

    # a~zまで始まる単語を空白ごとに区切ったリストをつくります
    letters_only = re.sub("[^a-zA-Z@]", " ",raw_tweet) 
    words = letters_only.lower().split()

    # '@'と'flight'が含まれる文字とストップワードを削除します
    stops = set(stopwords.words("english"))  
    meaningful_words = [w for w in words if not w in stops and not re.match("^[@]", w) and not re.match("flight",w)] 
    return( " ".join(meaningful_words)) 

cleanTweet = tweetData['text'].apply(lambda x: tweet_to_words(x))

# データベースを作成します
all_text = ' '.join(cleanTweet)
words = all_text.split()

# 単語の出現回数をカウントします
counts = Counter(words)

# 降順に並べ替えます
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}

# 新しいリストに文字列を数値化したものを格納します
tweet_ints = []
for each in cleanTweet:
    tweet_ints.append([vocab_to_int[word] for word in each.split()])

# Tweetのnegative/positiveの文字列を数字に変換します
labels = np.array([0 if each == 'negative' else 1 if each == 'positive' else 2 for each in tweetData['airline_sentiment'][:]]) 

# Tweetの単語数を調べます
tweet_len = Counter([len(x) for x in tweet_ints])
seq_len = max(tweet_len)

# cleanTweetで0文字になってしまった行を削除します
tweet_idx  = [idx for idx,tweet in enumerate(tweet_ints) if len(tweet) > 0]
labels = labels[tweet_idx]
tweet_ints = [tweet for tweet in tweet_ints if len(tweet) > 0]

# i行ごとの数値化した単語を右から書いたフレームワークを作ります。他の数字は0にします。
features = np.zeros((len(tweet_ints), seq_len), dtype=int)
for i, row in enumerate(tweet_ints):
    features[i, -len(row):] = np.array(row)[:seq_len]
    
    
    
#-----ここまでchapter2-----



# 元データから学習用データを作成
split_idx = int(len(features)*0.8)
train_x, val_x = features[:split_idx], features[split_idx:]
train_y, val_y = labels[:split_idx], labels[split_idx:]

# テスト用データを作成
test_idx = int(len(val_x)*0.5)
val_x, test_x = val_x[:test_idx], val_x[test_idx:]
val_y, test_y = val_y[:test_idx], val_y[test_idx:]

#学習データを(サンプル数, シーケンス長, 次元数) の形に変換する
train_x = np.reshape(train_x, (train_x.shape[0], 1, train_x.shape[1]))
test_x = np.reshape(test_x, (test_x.shape[0], 1, test_x.shape[1]))
val_x =  np.reshape(val_x, (val_x.shape[0], 1, val_x.shape[1]))
#正解ラベルをone-hotベクトルに変換
train_y = to_categorical(train_y, 3)
test_y = to_categorical(test_y, 3)
val_y = to_categorical(val_y, 3)

print("\t\t\tFeature Shapes:")
print("Train_x set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest_x set: \t\t{}".format(test_x.shape))
print("\n")
print("Train_y set: \t\t{}".format(train_y.shape), 
      "\nValidation set: \t{}".format(val_y.shape),
      "\nTest_y set: \t\t{}".format(test_y.shape))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/k-kakimoto/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


			Feature Shapes:
Train_x set: 		(11648, 1, 107) 
Validation set: 	(1456, 1, 107) 
Test_x set: 		(1457, 1, 107)


Train_y set: 		(11648, 3) 
Validation set: 	(1456, 3) 
Test_y set: 		(1457, 3)


## LSTM層の定義（1）
今回の実装ではKerasのSequentialモデルを使用</br>
Sequentialモデルではaddメソッドを使うことで、全結合層（Dense）などと同じように簡単にLSTM層を追加することができる</br>

今回は一層目にLSTM層（前に入力を引き継ぐため3次元）を定義するためinput_dim引数と、input_length引数を指定することで入力のshapeを指定</br>
また一層目がどんな層でも、input_shape引数にshapeを示すタプルを指定することで入力のshapeを指定することができる</br>
input_shape=(シークエンス長, 学習データの次元数)と指定

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,LSTM
from tensorflow.keras.callbacks import EarlyStopping

# chapter2のおさらい
import nltk
import numpy as np
import pandas as pd
import re
from collections import Counter
from nltk.corpus import stopwords
from tensorflow.keras.utils import to_categorical

# エラーが出たら↓をコメントアウトしてください
nltk.download('stopwords')

# Tweetデータを読み込みます
Tweet = pd.read_csv('./6020_negative_positive_data/data/Airline-Sentiment-2-w-AA.csv', encoding='cp932')
tweetData = Tweet.loc[:,['text','airline_sentiment']]

# 英語Tweetの形態素解析を行います
def tweet_to_words(raw_tweet):

    # a~zまで始まる単語を空白ごとに区切ったリストをつくります
    letters_only = re.sub("[^a-zA-Z@]", " ",raw_tweet) 
    words = letters_only.lower().split()

    # '@'と'flight'が含まれる文字とストップワードを削除します
    stops = set(stopwords.words("english"))  
    meaningful_words = [w for w in words if not w in stops and not re.match("^[@]", w) and not re.match("flight",w)] 
    return( " ".join(meaningful_words)) 

cleanTweet = tweetData['text'].apply(lambda x: tweet_to_words(x))

# データベースを作成します
all_text = ' '.join(cleanTweet)
words = all_text.split()

# 単語の出現回数をカウントします
counts = Counter(words)

# 降順に並べ替えます
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}

# 新しいリストに文字列を数値化したものを格納します
tweet_ints = []
for each in cleanTweet:
    tweet_ints.append([vocab_to_int[word] for word in each.split()])

# Tweetのnegative/positiveの文字列を数字に変換します
labels = np.array([0 if each == 'negative' else 1 if each == 'positive' else 2 for each in tweetData['airline_sentiment'][:]]) 

# Tweetの単語数を調べます
tweet_len = Counter([len(x) for x in tweet_ints])
seq_len = max(tweet_len)

# cleanTweetで0文字になってしまった行を削除します
tweet_idx  = [idx for idx,tweet in enumerate(tweet_ints) if len(tweet) > 0]
labels = labels[tweet_idx]
tweet_ints = [tweet for tweet in tweet_ints if len(tweet) > 0]

# i行ごとの数値化した単語を右から書いたフレームワークを作ります。他の数字は0にします。
features = np.zeros((len(tweet_ints), seq_len), dtype=int)
for i, row in enumerate(tweet_ints):
    features[i, -len(row):] = np.array(row)[:seq_len]
    
#-----ここまでchapter2-----

# 元データから学習用データを作成します
split_idx = int(len(features)*0.8)
train_x, val_x = features[:split_idx], features[split_idx:]
train_y, val_y = labels[:split_idx], labels[split_idx:]

# テスト用データを作成します
test_idx = int(len(val_x)*0.5)
val_x, test_x = val_x[:test_idx], val_x[test_idx:]
val_y, test_y = val_y[:test_idx], val_y[test_idx:]

# 学習データを(サンプル数, シーケンス長, 次元数) の形に変換する
train_x = np.reshape(train_x, (train_x.shape[0], 1,train_x.shape[1]))
test_x = np.reshape(test_x, (test_x.shape[0], 1 ,test_x.shape[1]))
val_x =  np.reshape(val_x, (val_x.shape[0], 1,val_x.shape[1]))
# 正解ラベルをone-hotベクトルに変換
train_y = to_categorical(train_y, 3)
test_y = to_categorical(test_y, 3)
val_y = to_categorical(val_y, 3)

lstm_model = Sequential()

# input_shapeのなかを埋めましょう
# input_shape=(シークエンス長, 学習データの次元数)
lstm_model.add(LSTM(units=60, input_shape=(train_x.shape[1], train_x.shape[2]) , return_sequences=True))

lstm_model.compile(loss='mean_squared_error', optimizer='Adagrad' , metrics = ['accuracy'])
lstm_model.summary()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/k-kakimoto/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1, 60)             40320     
                                                                 
Total params: 40,320
Trainable params: 40,320
Non-trainable params: 0
_________________________________________________________________


## LSTM層の定義（2）
2層目のLSTM層と、それに続く全結合層を定義</br>
LSTM層を定義する際、

- その層の次にもLSTM層を定義する場合はreturn_sequences引数にTrueを指定
- 次の層にLSTM層以外の層を定義する場合はFalseを指定


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, LSTM
from tensorflow.keras.callbacks import EarlyStopping

# chapter2のおさらい
import nltk
import numpy as np
import pandas as pd
import re
from collections import Counter
from nltk.corpus import stopwords
from tensorflow.keras.utils import to_categorical

# エラーが出たら↓をコメントアウトしてください
nltk.download('stopwords')

# Tweetデータを読み込みます
Tweet = pd.read_csv('./6020_negative_positive_data/data/Airline-Sentiment-2-w-AA.csv', encoding='cp932')
tweetData = Tweet.loc[:,['text','airline_sentiment']]

# 英語Tweetの形態素解析を行います
def tweet_to_words(raw_tweet):

    # a~zまで始まる単語を空白ごとに区切ったリストをつくります
    letters_only = re.sub("[^a-zA-Z@]", " ",raw_tweet) 
    words = letters_only.lower().split()

    # '@'と'flight'が含まれる文字とストップワードを削除します
    stops = set(stopwords.words("english"))  
    meaningful_words = [w for w in words if not w in stops and not re.match("^[@]", w) and not re.match("flight",w)] 
    return( " ".join(meaningful_words)) 

cleanTweet = tweetData['text'].apply(lambda x: tweet_to_words(x))

# データベースを作成します
all_text = ' '.join(cleanTweet)
words = all_text.split()

# 単語の出現回数をカウントします
counts = Counter(words)

# 降順に並べ替えます
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}

# 新しいリストに文字列を数値化したものを格納します
tweet_ints = []
for each in cleanTweet:
    tweet_ints.append([vocab_to_int[word] for word in each.split()])

# Tweetのnegative/positiveの文字列を数字に変換します
labels = np.array([0 if each == 'negative' else 1 if each == 'positive' else 2 for each in tweetData['airline_sentiment'][:]]) 

# Tweetの単語数を調べます
tweet_len = Counter([len(x) for x in tweet_ints])
seq_len = max(tweet_len)

# cleanTweetで0文字になってしまった行を削除します
tweet_idx  = [idx for idx,tweet in enumerate(tweet_ints) if len(tweet) > 0]
labels = labels[tweet_idx]
tweet_ints = [tweet for tweet in tweet_ints if len(tweet) > 0]

# i行ごとの数値化した単語を右から書いたフレームワークを作ります。他の数字は0にします。
features = np.zeros((len(tweet_ints), seq_len), dtype=int)
for i, row in enumerate(tweet_ints):
    features[i, -len(row):] = np.array(row)[:seq_len]
    
#-----ここまでchapter2-----

# 元データから学習用データを作成します
split_idx = int(len(features)*0.8)
train_x, val_x = features[:split_idx], features[split_idx:]
train_y, val_y = labels[:split_idx], labels[split_idx:]

# テスト用データを作成します
test_idx = int(len(val_x)*0.5)
val_x, test_x = val_x[:test_idx], val_x[test_idx:]
val_y, test_y = val_y[:test_idx], val_y[test_idx:]

# 学習データを(サンプル数, シーケンス長, 次元数) の形に変換する
train_x = np.reshape(train_x, (train_x.shape[0], 1,train_x.shape[1]))
test_x = np.reshape(test_x, (test_x.shape[0], 1 ,test_x.shape[1]))
val_x =  np.reshape(val_x, (val_x.shape[0], 1,val_x.shape[1]))
# 正解ラベルをone-hotベクトルに変換
train_y = to_categorical(train_y, 3)
test_y = to_categorical(test_y, 3)
val_y = to_categorical(val_y, 3)

lstm_model = Sequential()
lstm_model.add(LSTM(units=60, input_shape=(train_x.shape[1], train_x.shape[2]), return_sequences=True))

# 以下を埋めてください
lstm_model.add(LSTM(units=30, return_sequences=False))
# このモデルではネガティブかポジティブかニュートラルかどうかを判断します。
# このことをもとにunitsに指定する数を考えましょう
# なぜ3を指定するのか？
lstm_model.add(Dense(units=3, activation='softmax'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/k-kakimoto/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## コンパイル、学習の実行、モデルの評価
compileメソッドを用いて、モデルがどのような学習処理を設定</br>
ここでは3クラス分類なので損失関数にはcategorical_crossentropyを指定する</br>
またオプティマイザーにrmsprop、評価関数にaccuracyを指定すると以下のコードになる
```python
lstm_model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
```

fitメソッドを用いて学習を実行</br>
今回はエポック数2、バッチサイズを32
```python
lstm_model.fit(train_x, train_y, validation_data=(val_x, val_y), epochs=2, batch_size=32)
```

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,LSTM
from tensorflow.keras.callbacks import EarlyStopping

#chapter2のおさらい
import nltk
import numpy as np
import pandas as pd
import re
from collections import Counter
from nltk.corpus import stopwords
from tensorflow.keras.utils import to_categorical

# エラーが出たら↓をコメントアウトしてください
nltk.download('stopwords')

# Tweetデータを読み込みます
Tweet = pd.read_csv('./6020_negative_positive_data/data/Airline-Sentiment-2-w-AA.csv', encoding='cp932')
tweetData = Tweet.loc[:,['text','airline_sentiment']]

# 英語Tweetの形態素解析を行います
def tweet_to_words(raw_tweet):

    # a~zまで始まる単語を空白ごとに区切ったリストをつくります
    letters_only = re.sub("[^a-zA-Z@]", " ",raw_tweet) 
    words = letters_only.lower().split()

    # '@'と'flight'が含まれる文字とストップワードを削除します
    stops = set(stopwords.words("english"))  
    meaningful_words = [w for w in words if not w in stops and not re.match("^[@]", w) and not re.match("flight",w)] 
    return( " ".join(meaningful_words)) 

cleanTweet = tweetData['text'].apply(lambda x: tweet_to_words(x))

# データベースを作成します
all_text = ' '.join(cleanTweet)
words = all_text.split()

# 単語の出現回数をカウントします
counts = Counter(words)

# 降順に並べ替えます
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}

# 新しいリストに文字列を数値化したものを格納します
tweet_ints = []
for each in cleanTweet:
    tweet_ints.append([vocab_to_int[word] for word in each.split()])

# Tweetのnegative/positiveの文字列を数字に変換します
labels = np.array([0 if each == 'negative' else 1 if each == 'positive' else 2 for each in tweetData['airline_sentiment'][:]]) 

# Tweetの単語数を調べます
tweet_len = Counter([len(x) for x in tweet_ints])
seq_len = max(tweet_len)

# cleanTweetで0文字になってしまった行を削除します
tweet_idx  = [idx for idx,tweet in enumerate(tweet_ints) if len(tweet) > 0]
labels = labels[tweet_idx]
tweet_ints = [tweet for tweet in tweet_ints if len(tweet) > 0]

# i行ごとの数値化した単語を右から書いたフレームワークを作ります。他の数字は0にします。
features = np.zeros((len(tweet_ints), seq_len), dtype=int)
for i, row in enumerate(tweet_ints):
    features[i, -len(row):] = np.array(row)[:seq_len]
    
#-----ここまでchapter2-----

# 元データから学習用データを作成します
split_idx = int(len(features)*0.8)
train_x, val_x = features[:split_idx], features[split_idx:]
train_y, val_y = labels[:split_idx], labels[split_idx:]

# テスト用データを作成します
test_idx = int(len(val_x)*0.5)
val_x, test_x = val_x[:test_idx], val_x[test_idx:]
val_y, test_y = val_y[:test_idx], val_y[test_idx:]

#学習データを(サンプル数, シーケンス長, 次元数) の形に変換する
train_x = np.reshape(train_x, (train_x.shape[0], 1,train_x.shape[1]))
test_x = np.reshape(test_x, (test_x.shape[0], 1 ,test_x.shape[1]))
val_x =  np.reshape(val_x, (val_x.shape[0], 1,val_x.shape[1]))
#正解ラベルをone-hotベクトルに変換
train_y = to_categorical(train_y, 3)
test_y = to_categorical(test_y, 3)
val_y = to_categorical(val_y, 3)

lstm_model = Sequential()
lstm_model.add(LSTM(units=64, input_shape=(train_x.shape[1], train_x.shape[2]), return_sequences=True))
lstm_model.add(LSTM(units=32, return_sequences=False))
lstm_model.add(Dense(units=3,activation='softmax'))

lstm_model.compile(optimizer="rmsprop" , # オプティマイザーを指定してください。
              loss="categorical_crossentropy", # 損失関数を指定してください。
              metrics="accuracy") # 評価関数を指定してください。

lstm_model.fit(train_x, train_y, validation_data=(val_x, val_y), epochs=2, batch_size=32)

score = lstm_model.evaluate(test_x, test_y, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/k-kakimoto/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2021-12-28 00:56:30.068705: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/2
364/364 [==============================] - 2s 2ms/step - loss: 0.9368 - accuracy: 0.6032 - val_loss: 0.8521 - val_accuracy: 0.6738
Epoch 2/2
46/46 [==============================] - 0s 526us/step - loss: 0.7264 - accuracy: 0.7639
Test loss: 0.7264183163642883
Test accuracy: 0.7638984322547913


## 入力データの加工
先ほど学習させたモデルで新たなテキストのネガポジ判定を実装</br>
モデルに入力するテキストデータは、モデルが学習したデータの形と同じである必要がある

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,LSTM
from tensorflow.keras.callbacks import EarlyStopping

#chapter2のおさらい
import nltk
import numpy as np
import pandas as pd
import re
from collections import Counter
from nltk.corpus import stopwords
from tensorflow.keras.utils import to_categorical

# エラーが出たら↓をコメントアウトしてください
nltk.download('stopwords')

# Tweetデータを読み込みます
Tweet = pd.read_csv('./6020_negative_positive_data/data/Airline-Sentiment-2-w-AA.csv', encoding='cp932')
tweetData = Tweet.loc[:,['text','airline_sentiment']]

# 英語Tweetの形態素解析を行います
def tweet_to_words(raw_tweet):

    # a~zまで始まる単語を空白ごとに区切ったリストをつくります
    letters_only = re.sub("[^a-zA-Z@]", " ",raw_tweet) 
    words = letters_only.lower().split()

    # '@'と'flight'が含まれる文字とストップワードを削除します
    stops = set(stopwords.words("english"))  
    meaningful_words = [w for w in words if not w in stops and not re.match("^[@]", w) and not re.match("flight",w)] 
    return( " ".join(meaningful_words)) 

cleanTweet = tweetData['text'].apply(lambda x: tweet_to_words(x))

# データベースを作成します
all_text = ' '.join(cleanTweet)
words = all_text.split()

# 単語の出現回数をカウントします
counts = Counter(words)

# 降順に並べ替えます
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}

# 新しいリストに文字列を数値化したものを格納します
tweet_ints = []
for each in cleanTweet:
    tweet_ints.append([vocab_to_int[word] for word in each.split()])

# Tweetのnegative/positiveの文字列を数字に変換します
labels = np.array([0 if each == 'negative' else 1 if each == 'positive' else 2 for each in tweetData['airline_sentiment'][:]]) 

# Tweetの単語数を調べます
tweet_len = Counter([len(x) for x in tweet_ints])
seq_len = max(tweet_len)

# cleanTweetで0文字になってしまった行を削除します
tweet_idx  = [idx for idx,tweet in enumerate(tweet_ints) if len(tweet) > 0]
labels = labels[tweet_idx]
tweet_ints = [tweet for tweet in tweet_ints if len(tweet) > 0]

# i行ごとの数値化した単語を右から書いたフレームワークを作ります。他の数字は0にします。
features = np.zeros((len(tweet_ints), seq_len), dtype=int)
for i, row in enumerate(tweet_ints):
    features[i, -len(row):] = np.array(row)[:seq_len]
    
#-----ここまでchapter2-----

# 元データから学習用データを作成します
split_idx = int(len(features)*0.8)
train_x, val_x = features[:split_idx], features[split_idx:]
train_y, val_y = labels[:split_idx], labels[split_idx:]

# テスト用データを作成します
test_idx = int(len(val_x)*0.5)
val_x, test_x = val_x[:test_idx], val_x[test_idx:]
val_y, test_y = val_y[:test_idx], val_y[test_idx:]

# 学習データを(サンプル数, シーケンス長, 次元数) の形に変換する
train_x = np.reshape(train_x, (train_x.shape[0], 1,train_x.shape[1]))
test_x = np.reshape(test_x, (test_x.shape[0], 1 ,test_x.shape[1]))
val_x =  np.reshape(val_x, (val_x.shape[0], 1,val_x.shape[1]))
# 正解ラベルをone-hotベクトルに変換
train_y = to_categorical(train_y, 3)
test_y = to_categorical(test_y, 3)
val_y = to_categorical(val_y, 3)

text = "it was amazing, and arrived an hour early. You're too good to me."

# 英語textの形態素解析を行い、データベースを作成します。
text = tweet_to_words(text)
text = text.split()

# 入力された単語を、学習データと同じように数値に変換します
# 学習したデータベースにない単語は0としています
words_int = []
for word in text:
    try:
        words_int.append(vocab_to_int[word])
    except :
        words_int.append(0)

# モデルが学習したデータの形に加工します
features = np.zeros((1, seq_len), dtype=int)
features[0][-len(words_int):] = words_int
features = np.reshape(features, (1, features.shape[0], features.shape[1]))


print(features)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/k-kakimoto/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
     0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
     0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
     0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
     0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
     0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   244 433  26 239  57]]]


## 入力データのネガポジ予測
加工済みの入力データをモデルに入力して、入力データのネガポジを予測

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,LSTM
from tensorflow.keras.callbacks import EarlyStopping

#chapter2のおさらい
import nltk
import numpy as np
import pandas as pd
import re
from collections import Counter
from nltk.corpus import stopwords
from tensorflow.keras.utils import to_categorical

# エラーが出たら↓をコメントアウトしてください
nltk.download('stopwords')

# Tweetデータを読み込みます
Tweet = pd.read_csv('./6020_negative_positive_data/data/Airline-Sentiment-2-w-AA.csv', encoding='cp932')
tweetData = Tweet.loc[:,['text','airline_sentiment']]

# 英語Tweetの形態素解析を行います
def tweet_to_words(raw_tweet):

    # a~zまで始まる単語を空白ごとに区切ったリストをつくります
    letters_only = re.sub("[^a-zA-Z@]", " ",raw_tweet)
    words = letters_only.lower().split()

    # '@'と'flight'が含まれる文字とストップワードを削除します
    stops = set(stopwords.words("english"))
    meaningful_words = [w for w in words if not w in stops and not re.match("^[@]", w) and not re.match("flight",w)]
    return(" ".join(meaningful_words))

cleanTweet = tweetData['text'].apply(lambda x: tweet_to_words(x))

# データベースを作成します
all_text = ' '.join(cleanTweet)
words = all_text.split()

# 単語の出現回数をカウントします
counts = Counter(words)

# 降順に並べ替えます
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}

# 新しいリストに文字列を数値化したものを格納します
tweet_ints = []
for each in cleanTweet:
    tweet_ints.append([vocab_to_int[word] for word in each.split()])

# Tweetのnegative/positiveの文字列を数字に変換します
labels = np.array([0 if each == 'negative' else 1 if each == 'positive' else 2 for each in tweetData['airline_sentiment'][:]])

# Tweetの単語数を調べます
tweet_len = Counter([len(x) for x in tweet_ints])
seq_len = max(tweet_len)

# cleanTweetで0文字になってしまった行を削除します
tweet_idx  = [idx for idx,tweet in enumerate(tweet_ints) if len(tweet) > 0]
labels = labels[tweet_idx]
tweet_ints = [tweet for tweet in tweet_ints if len(tweet) > 0]

# i行ごとの数値化した単語を右から書いたフレームワークを作ります。他の数字は0にします。
features = np.zeros((len(tweet_ints), seq_len), dtype=int)
for i, row in enumerate(tweet_ints):
    features[i, -len(row):] = np.array(row)[:seq_len]

#-----ここまでchapter2-----

# 元データから学習用データを作成します
split_idx = int(len(features)*0.8)
train_x, val_x = features[:split_idx], features[split_idx:]
train_y, val_y = labels[:split_idx], labels[split_idx:]

# テスト用データを作成します
test_idx = int(len(val_x)*0.5)
val_x, test_x = val_x[:test_idx], val_x[test_idx:]
val_y, test_y = val_y[:test_idx], val_y[test_idx:]

# 学習データを(サンプル数, シーケンス長, 次元数) の形に変換する
train_x = np.reshape(train_x, (train_x.shape[0], 1,train_x.shape[1]))
test_x = np.reshape(test_x, (test_x.shape[0], 1 ,test_x.shape[1]))
val_x =  np.reshape(val_x, (val_x.shape[0], 1,val_x.shape[1]))
# 正解ラベルをone-hotベクトルに変換
train_y = to_categorical(train_y, 3)
test_y = to_categorical(test_y, 3)
val_y = to_categorical(val_y, 3)

negaposi = ["negative","positive","neutral"]

text = "The plane was constantly shaking due to bad weather and it was not very comfortable."

# 英語textの形態素解析を行い、データベースを作成します。
text = tweet_to_words(text)
text = text.split()

# 入力された単語を、学習データと同じように数値に変換します
# 学習したデータベースにない単語は0としています
words_int = []
for word in text:
    try:
        words_int.append(vocab_to_int[word])
    except :
        words_int.append(0)

# モデルが学習したデータの形に加工します
features = np.zeros((1, seq_len), dtype=int)
features[0][-len(words_int):] = words_int
features = np.reshape(features, (1, features.shape[0], features.shape[1]))

# モデルの構築
lstm_model = Sequential()
lstm_model.add(LSTM(units=64, input_shape=(train_x.shape[1], train_x.shape[2]), return_sequences=True))
lstm_model.add(LSTM(units=32, return_sequences=False))
lstm_model.add(Dense(units=3,activation='softmax'))

lstm_model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

lstm_model.fit(train_x, train_y, validation_data=(val_x, val_y), epochs=2, batch_size=32)
print()

# 入力データのネガポジを予測します
predict = lstm_model.predict([features])
answer = np.argmax(predict) # こちらを記入してください
print(predict)
print("pred answer: ", negaposi[answer])

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/k-kakimoto/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Epoch 1/2
364/364 [==============================] - 2s 3ms/step - loss: 0.9336 - accuracy: 0.6029 - val_loss: 0.8469 - val_accuracy: 0.6731
Epoch 2/2
364/364 [==============================] - 0s 944us/step - loss: 0.9079 - accuracy: 0.6144 - val_loss: 0.8293 - val_accuracy: 0.6799

[[0.6469767  0.12680197 0.22622137]]
pred answer:  negative
